In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import json
import time
import csv
import pandas as pd

In [2]:
csvFile = open("namelist.csv","w+")
writer = csv.writer(csvFile)
writer.writerow(('name','url'))
urlList = []

In [3]:
sourceUrl = "https://sf.taobao.com/item_list.htm?category=50025969&auction_source=0&province=%CC%EC%BD%F2&sorder=2" \
                 "&st_param=-1&auction_start_from=2018-01-01&auction_start_to=2019-05-13&spm=a213w.3064813.9001.2"  



In [4]:
# Start the driver
driver = webdriver.Chrome()
# Setup wait for later
wait = WebDriverWait(driver, 10)

driver.get(sourceUrl)

# Store the ID of the original window
original_window = driver.current_window_handle

# Check we don't have other windows open already
assert len(driver.window_handles) == 1

In [5]:
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "page-total")))

pages = driver.find_element_by_class_name("page-total").text


In [6]:
#for j in range(1,int(pages)+1):
for j in range(1,2):

    if (j > 1):
        driver.find_element_by_link_text(str(j)).click()
        time.sleep(1)
        #WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "current")))

    else:
        pass
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "sf-item-list-data")))
    data = driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML')
    jsonData =  json.loads(data)
    jd = jsonData["data"]
    
    
    
    
    #for i in range(len(jd)):
    for i in range(2):
        num = jd[i]["id"]
        url = "#pai-item-"+str(num)+" .pic"
        urlList.append(url)
        # Click the link which opens in a new window
        driver.find_element_by_css_selector(url).click()
        # Wait for the new window or tab
        wait.until(EC.number_of_windows_to_be(2))
        
        handles = driver.window_handles
        #print(handles)
        driver.switch_to.window(handles[1])
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "bid-fail")))
        data = driver.find_element_by_class_name("bid-fail").text
        bidStatus = data[5:7]
        if(bidStatus !="结束"):
            print("流拍 or 失败")
            print(str(j)+":"+str(i) +":"+ data+driver.title)
            print("url"+driver.current_url)
        else:
            url2 = driver.current_url+"#J_Confirmation"
            driver.get(url2)
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "c-content")))
            info = driver.find_element_by_class_name("c-content").text
            index = info.find("通过")
            name = info[4:index]
            print("结束"+":"+name)
            print(str(j)+":"+str(i) +":"+ data+driver.title)
            print("url:"+driver.current_url)
            
            # add to file
            writer.writerow((name,driver.current_url))
            

        driver.close()
        driver.switch_to.window(handles[0])


结束:杨宁
1:0:本场拍卖已结束 ！天津市西青区泰佳道与汇川路交口东南侧武台馨苑13-1-2205号 - 司法拍卖 - 阿里拍卖
url:https://sf-item.taobao.com/sf_item/591345585914.htm?spm=a213w.7398504.paiList.1.4b4c26cdIV7YFs#J_Confirmation
结束:魏环利
1:1:本场拍卖已结束 ！天津市和平区西安道与长沙路交口东北侧诚基经贸中心1-2-902 - 司法拍卖 - 阿里拍卖
url:https://sf-item.taobao.com/sf_item/593056639808.htm?spm=a213w.7398504.paiList.2.4b4c26cdIV7YFs#J_Confirmation


In [7]:
driver.quit()

In [8]:
csvFile.close()
csv = pd.read_csv('namelist.csv', encoding='utf-8')
csv.to_excel('namelist.xlsx', sheet_name='data')

In [9]:
#for i in range(len(urlList))


#     driver.switchTo().window("vars.get("root").toString()");
#     driver.findElement(By.cssSelector("#pai-item-591508706185 .pic")).click();
#     driver.switchTo().window("vars.get("win8140").toString()");
#     driver.close();

In [10]:
#urlList

In [11]:
#590871909472
#pai-item-591508706185 .pic

In [12]:
#data = driver.find_element_by_class_name("bid-fail").text
#print(data)

In [13]:
#d = driver.find_element_by_id("sf-item-list-data").get_attribute('innerHTML')


In [14]:
#jd = json.loads(d)


In [15]:
# for i in range(len(jd["data"])):
#     print(jd["data"][i]["id"])

In [16]:
#driver.find_element_by_css_selector(url).click()